In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def calc_remains(ser, paid):
    remain = ser["стоимость"] - ser.первоначальный_взнос
    remain_arr = [remain]
    while remain>0:
        remain = (remain - paid) * (1+0.004167)
        remain_arr.append(remain)
    return remain_arr

def generate_payments_df(ser):
    payments = pd.DataFrame({"remain":calc_remains(ser, плата_по_кредиту)})
    payments = payments.reset_index()
    payments.columns = ["month", "remain"]
    payments["credit_payment"] = плата_по_кредиту
    payments.loc[0, "first_pay"] = ser.первоначальный_взнос
    payments.loc[0:ser.можно_заехать_через_месяцев-1, "repair_payments"] = ser.стоимость_ремонта/ser.можно_заехать_через_месяцев
    payments.loc[0:ser.можно_заехать_через_месяцев-1, "rent_payments"] = стоимость_аренды
    payments = payments.fillna(0)
    
    payments["cum_first_pay"] = payments["first_pay"].cumsum()
    payments["cum_credit_payment"] = payments["credit_payment"].cumsum()
    payments["cum_repair_payments"] = payments["repair_payments"].cumsum()
    payments["cum_rent_payments"] = payments["rent_payments"].cumsum()
    
    
    payments["cum_payment"] = payments["cum_first_pay"]+payments["cum_credit_payment"]+payments["cum_repair_payments"]+payments["cum_rent_payments"]
    payments["year"] = payments["month"]/12
    return payments


In [ ]:
def make_report(df):
    df["ставка_месячная"] = df["ставка"]/12
    df["выплаченная_аренда_до_времени_заезда"] = df["можно_заехать_через_месяцев"] * стоимость_аренды
    df["первоначальный_взнос"] = сумма_на_руках - df["стоимость_ремонта"]

    ser1 = df.iloc[0]
    payments_1 = generate_payments_df(ser1)
    ser2 = df.iloc[1]
    payments_2 = generate_payments_df(ser2)
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True,
        subplot_titles=[
        f"{ser1.тип} за {round(ser1.стоимость/1e6, 1)} млн, с ремонтом за {ser1.стоимость_ремонта/1e6} млн и заселением через {round(ser1.можно_заехать_через_месяцев/12,1)} лет.  <br> sum={round(payments_1.cum_payment.iloc[-1]/1e6, 1)} млн",
        f"{ser2.тип} за {round(ser2.стоимость/1e6, 1)} млн, с ремонтом за {ser2.стоимость_ремонта/1e6} млн и заселением через {round(ser2.можно_заехать_через_месяцев/12,1)} лет. <br> sum={round(payments_2.cum_payment.iloc[-1]/1e6, 1)} млн"
        ])

    fig.add_trace(go.Scatter(x=payments_1.year, y=payments_1.cum_first_pay, mode='lines', stackgroup='one', name="cum_first_pay", line_color="#0d0887"),row=1, col=1)
    fig.add_trace(go.Scatter(x=payments_1.year, y=payments_1.cum_repair_payments, mode='lines', stackgroup='one', name="cum_repair_payments", line_color="#1f77b4"),row=1, col=1)
    fig.add_trace(go.Scatter(x=payments_1.year, y=payments_1.cum_rent_payments, mode='lines', stackgroup='one', name="cum_rent_payments", line_color="#ff7f0e"),row=1, col=1)
    fig.add_trace(go.Scatter(x=payments_1.year, y=payments_1.cum_credit_payment, mode='lines', stackgroup='one', name="cum_credit_payment", line_color="#2ca02c"),row=1, col=1)

    fig.add_trace(go.Scatter(x=payments_2.year, y=payments_2.cum_first_pay, mode='lines', stackgroup='one', name="cum_first_pay", line_color="#0d0887"),row=1, col=2)
    fig.add_trace(go.Scatter(x=payments_2.year, y=payments_2.cum_repair_payments, mode='lines', stackgroup='one', name="cum_repair_payments", line_color="#1f77b4"),row=1, col=2)
    fig.add_trace(go.Scatter(x=payments_2.year, y=payments_2.cum_rent_payments, mode='lines', stackgroup='one', name="cum_rent_payments", line_color="#ff7f0e"),row=1, col=2)
    fig.add_trace(go.Scatter(x=payments_2.year, y=payments_2.cum_credit_payment, mode='lines', stackgroup='one', name="cum_credit_payment", line_color="#2ca02c"),row=1, col=2)


    fig.update_layout(height=500, xaxis_title="Месяц", yaxis_title="Кумулятивная плата",)
    fig.show()
    return payments_1, payments_2

In [ ]:
сумма_на_руках = 5_000_000
стоимость_аренды = 50_000
плата_по_кредиту = 100_000

var_1 = {"тип":"неоч вторичка", "стоимость": 16_000_000, "ставка": 0.10, "стоимость_ремонта": 2_000_000, "можно_заехать_через_месяцев": 6}
var_2 = {"тип":"нормальная вторичка", "стоимость": 18_000_000, "ставка": 0.10, "стоимость_ремонта": 500_000, "можно_заехать_через_месяцев": 1}
df = pd.DataFrame([var_1,var_2])
payments_1, payments_2 = make_report(df)